# k-Means Clustering mit künstlichen Daten

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_19/artificial_clustering_kmeans.ipynb)

In diesem Notebook lernen wir k-Means mit **perfekten** Daten kennen. Wir erzeugen drei klar getrennte Cluster und schauen, wie gut k-Means diese findet.

**Ziel:** Verstehen, wie k-Means unter idealen Bedingungen funktioniert und warum Skalierung auch hier wichtig sein kann.

## 1. Bibliotheken importieren

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, silhouette_score

print("Alle Bibliotheken geladen! 🚀")

## 2. Künstliche Daten mit verschiedenen Skalen erzeugen

Wir erstellen zwei Versionen: eine mit ähnlichen Skalen, eine mit sehr verschiedenen Skalen.

In [ ]:
# Zufallszahlen für Reproduzierbarkeit festlegen
np.random.seed(42)

# Version 1: Drei gut getrennte Cluster mit ähnlichen Skalen
X_similar, y_true = make_blobs(
    n_samples=300,          # 300 Datenpunkte
    centers=3,              # 3 Cluster
    n_features=2,           # 2 Dimensionen (x, y)
    cluster_std=1.5,        # Streuung innerhalb der Cluster
    center_box=(-5, 5),     # Zentren zwischen -5 und +5
    random_state=42
)

print("📊 Ähnliche Skalen - Dateneigenschaften:")
print(f"Feature 1 (X): {X_similar[:, 0].min():.1f} bis {X_similar[:, 0].max():.1f}")
print(f"Feature 2 (Y): {X_similar[:, 1].min():.1f} bis {X_similar[:, 1].max():.1f}")
print(f"Anzahl Punkte pro Cluster: {np.bincount(y_true)}")

# Version 2: Gleiche Cluster, aber Feature 2 stark vergrößert (Skalierungsproblem!)
X_different = X_similar.copy()
X_different[:, 1] = X_different[:, 1] * 100  # Y-Werte 100x größer machen!

print("\n⚠️ Verschiedene Skalen - Dateneigenschaften:")
print(f"Feature 1 (X): {X_different[:, 0].min():.1f} bis {X_different[:, 0].max():.1f}")
print(f"Feature 2 (Y): {X_different[:, 1].min():.0f} bis {X_different[:, 1].max():.0f}")
print("🚨 Riesiger Unterschied! Y ist 100x größer als X!")

## 3. Die perfekten Cluster visualisieren

Schauen wir uns beide Versionen an:

In [ ]:
# Beide Versionen nebeneinander plotten - exakt gleiche Breite
fig, axes = plt.subplots(1, 2, figsize=(15, 6), gridspec_kw={'width_ratios': [1, 1]})

colors = ['red', 'blue', 'green']
cluster_names = ['Cluster A', 'Cluster B', 'Cluster C']

# Plot 1: Ähnliche Skalen
for i in range(3):
    mask = y_true == i
    axes[0].scatter(X_similar[mask, 0], X_similar[mask, 1], 
                   c=colors[i], label=cluster_names[i], alpha=0.7, s=50)

axes[0].set_xlabel('Feature 1 (X)')
axes[0].set_ylabel('Feature 2 (Y)')
axes[0].set_title('Ähnliche Skalen\n(Beide Features gleich wichtig)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Verschiedene Skalen
for i in range(3):
    mask = y_true == i
    axes[1].scatter(X_different[mask, 0], X_different[mask, 1], 
                   c=colors[i], label=cluster_names[i], alpha=0.7, s=50)

axes[1].set_xlabel('Feature 1 (X)')
axes[1].set_ylabel('Feature 2 (Y)')
axes[1].set_title('Verschiedene Skalen\n(Y dominiert)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Beobachtungen:")
print("• Links: Drei schöne, runde Cluster - perfekt für k-Means!")
print("• Rechts: Gleiche Cluster, aber Y-Achse dominiert völlig (schau auf die Zahlen an der y-Achse!)")
print("• Ohne Skalierung würde k-Means nur die Y-Unterschiede sehen!")

## 4. k-Means ohne Skalierung (verschiedene Skalen)

Zuerst testen wir k-Means mit den unskalierten Daten:

In [ ]:
# k-Means auf verschiedene Skalen anwenden (OHNE Skalierung)
print("🤖 k-Means OHNE Skalierung (auf verschiedene Skalen):")

kmeans_unscaled = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_unscaled = kmeans_unscaled.fit_predict(X_different)
centers_unscaled = kmeans_unscaled.cluster_centers_

# Ergebnisse bewerten
ari_unscaled = adjusted_rand_score(y_true, labels_unscaled)
sil_unscaled = silhouette_score(X_different, labels_unscaled)

print(f"Adjusted Rand Index: {ari_unscaled:.3f}")
print(f"Silhouette Score: {sil_unscaled:.3f}")

# Visualisierung
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_different[:, 0], X_different[:, 1], c=labels_unscaled, 
                     cmap='viridis', alpha=0.7, s=50)
plt.scatter(centers_unscaled[:, 0], centers_unscaled[:, 1], 
           c='red', marker='x', s=200, linewidths=3, label='k-Means Zentren')

plt.xlabel('Feature 1 (X)')
plt.ylabel('Feature 2 (Y) - 100x vergrößert')
plt.title('OHNE Skalierung: k-Means\n(Y-Achse dominiert)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.colorbar(scatter, label='k-Means Cluster')
plt.show()

print("\n📊 Bewertung OHNE Skalierung:")
print(f"• ARI = {ari_unscaled:.3f}: {'Perfekt!' if ari_unscaled > 0.9 else 'Gut' if ari_unscaled > 0.6 else 'Mäßig' if ari_unscaled > 0.4 else 'Schlecht'}")
print(f"• Silhouette = {sil_unscaled:.3f}: {'Sehr gut!' if sil_unscaled > 0.7 else 'Gut' if sil_unscaled > 0.5 else 'Mäßig' if sil_unscaled > 0.3 else 'Schlecht'}")

## 5. Daten skalieren

Jetzt skalieren wir die Daten und schauen was passiert.

In [ ]:
# Daten skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_different)

print("Nach der Skalierung:")
print(f"Feature 1: Mittelwert={X_scaled[:, 0].mean():.3f}, Std={X_scaled[:, 0].std():.3f}")
print(f"Feature 2: Mittelwert={X_scaled[:, 1].mean():.3f}, Std={X_scaled[:, 1].std():.3f}")
print("\n🎯 Jetzt sind beide Features gleichberechtigt!")

# Skalierte Daten visualisieren
plt.figure(figsize=(10, 6))
for i in range(3):
    mask = y_true == i
    plt.scatter(X_scaled[mask, 0], X_scaled[mask, 1], 
               c=colors[i], label=cluster_names[i], alpha=0.7, s=50)

plt.xlabel('Feature 1 (skaliert)')
plt.ylabel('Feature 2 (skaliert)')
plt.title('Skalierte Daten: Beide Features gleichberechtigt')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')  # Gleiche Skala für beide Achsen
plt.show()


## 6. k-Means mit Skalierung

Jetzt wenden wir k-Means auf die skalierten Daten an:

In [ ]:
# k-Means auf skalierte Daten anwenden
print("🤖 k-Means MIT Skalierung:")

kmeans_scaled = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_scaled = kmeans_scaled.fit_predict(X_scaled)
centers_scaled = kmeans_scaled.cluster_centers_

print("Cluster-Zentren (skaliert):")
for i, center in enumerate(centers_scaled):
    print(f"  Cluster {i}: [{center[0]:.2f}, {center[1]:.2f}]")

# Wie viele Punkte pro Cluster?
unique, counts = np.unique(labels_scaled, return_counts=True)
print("\n📊 Anzahl Punkte pro k-Means Cluster:")
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} Punkte")

print("\n🎯 Zur Erinnerung - echte Verteilung:")
unique_real, counts_real = np.unique(y_true, return_counts=True)
for cluster, count in zip(unique_real, counts_real):
    print(f"  Echter Cluster {cluster}: {count} Punkte")

## 7. Ergebnisse visualisieren und vergleichen

Schauen wir uns alle Varianten nebeneinander an:

In [ ]:
# Drei Plots nebeneinander: Original, ohne Skalierung, mit Skalierung
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Echte Cluster (auf Original-Skala)
for i in range(3):
    mask = y_true == i
    axes[0].scatter(X_different[mask, 0], X_different[mask, 1], 
                   c=colors[i], label=cluster_names[i], alpha=0.7, s=50)
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2 (100x)')
axes[0].set_title('Echte Cluster')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: k-Means ohne Skalierung
scatter1 = axes[1].scatter(X_different[:, 0], X_different[:, 1], 
                          c=labels_unscaled, cmap='viridis', alpha=0.7, s=50)
axes[1].scatter(centers_unscaled[:, 0], centers_unscaled[:, 1], 
               c='red', marker='x', s=200, linewidths=3, label='Zentren')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2 (100x)')
axes[1].set_title('k-Means OHNE Skalierung')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[1], label='Cluster')

# Plot 3: k-Means mit Skalierung (zurück-transformiert für Anzeige)
centers_orig = scaler.inverse_transform(centers_scaled)
scatter2 = axes[2].scatter(X_different[:, 0], X_different[:, 1], 
                          c=labels_scaled, cmap='viridis', alpha=0.7, s=50)
axes[2].scatter(centers_orig[:, 0], centers_orig[:, 1], 
               c='red', marker='x', s=200, linewidths=3, label='Zentren')
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2 (100x)')
axes[2].set_title('k-Means MIT Skalierung')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[2], label='Cluster')

plt.tight_layout()
plt.show()

print("👀 Beobachtungen:")
print("• Links: Die echten, perfekt getrennten Cluster")
print("• Mitte: k-Means ohne Skalierung - fokussiert nur auf Y-Achse")
print("• Rechts: k-Means mit Skalierung - berücksichtigt beide Achsen fair")

## 8. Metriken vergleichen

Jetzt bewerten wir beide Ansätze quantitativ:

In [ ]:
# Metriken für skalierte Version berechnen
ari_scaled = adjusted_rand_score(y_true, labels_scaled)
sil_scaled = silhouette_score(X_scaled, labels_scaled)

print("📊 Detaillierte Bewertung:")
print()
print("🔍 Adjusted Rand Index (ARI) - Vergleich mit echten Clustern:")
print(f"• OHNE Skalierung: {ari_unscaled:.3f}")
print(f"• MIT Skalierung:  {ari_scaled:.3f}")
print(f"• Verbesserung:     {((ari_scaled - ari_unscaled) / ari_unscaled * 100):+.1f}%" if ari_unscaled > 0 else "• Deutlicher Unterschied!")
print()
print("🔍 Silhouette Score - Cluster-Kompaktheit:")
print(f"• OHNE Skalierung: {sil_unscaled:.3f}")
print(f"• MIT Skalierung:  {sil_scaled:.3f}")
print(f"• Verbesserung:     {((sil_scaled - sil_unscaled) / sil_unscaled * 100):+.1f}%" if sil_unscaled > 0 else "• Deutlicher Unterschied!")

print("\n📋 Zusammenfassung:")
print(f"{'Methode':<20} {'ARI':<8} {'Silhouette':<12} {'Qualität':<15}")
print("-" * 55)
ari_quality = "Perfekt!" if ari_unscaled > 0.9 else "Sehr gut" if ari_unscaled > 0.7 else "Gut" if ari_unscaled > 0.5 else "Mäßig"
print(f"{'Ohne Skalierung':<20} {ari_unscaled:.3f}{'':>4} {sil_unscaled:.3f}{'':>8} {ari_quality}")
ari_quality2 = "Perfekt!" if ari_scaled > 0.9 else "Sehr gut" if ari_scaled > 0.7 else "Gut" if ari_scaled > 0.5 else "Mäßig"
print(f"{'Mit Skalierung':<20} {ari_scaled:.3f}{'':>4} {sil_scaled:.3f}{'':>8} {ari_quality2}")

print("\n🎯 Skalierung verbessert die Ergebnisse!")


## Vertraue nicht blind den Metriken!

Der Silhouette-Score auf den unskalierten Daten ist etwas höher (weil die Cluster-Grenzen dort "einfacher" bzw. "eindeutiger" sind). Aber aufgrund der fehlenden Skalierung wird das zweite Merkmal hier ignoriert! Die Cluster auf den skalierten Daten sind sinnvoller! Der kleine Unterschied im Silhouette-Score ist hier nicht signifikant!

## 9. Bonus: k-Means auf den ursprünglichen (ähnlichen) Skalen

Zum Vergleich schauen wir uns auch an, wie k-Means mit den ursprünglich ähnlichen Skalen funktioniert:

In [ ]:
# k-Means auf die ursprünglichen Daten mit ähnlichen Skalen
print("🎯 Bonus: k-Means auf ähnliche Skalen (Idealsituation):")

kmeans_ideal = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_ideal = kmeans_ideal.fit_predict(X_similar)

ari_ideal = adjusted_rand_score(y_true, labels_ideal)
sil_ideal = silhouette_score(X_similar, labels_ideal)

print(f"ARI: {ari_ideal:.3f} - {'Perfekt!' if ari_ideal > 0.9 else 'Sehr gut!'}")
print(f"Silhouette: {sil_ideal:.3f} - {'Sehr gut!' if sil_ideal > 0.7 else 'Gut!'}")

# Visualisierung
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_similar[:, 0], X_similar[:, 1], c=labels_ideal, 
                     cmap='viridis', alpha=0.7, s=50)
plt.scatter(kmeans_ideal.cluster_centers_[:, 0], kmeans_ideal.cluster_centers_[:, 1], 
           c='red', marker='x', s=200, linewidths=3, label='k-Means Zentren')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('k-Means auf ähnliche Skalen\n(Idealsituation - keine Skalierung nötig)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.colorbar(scatter, label='k-Means Cluster')
plt.axis('equal')
plt.show()

print("\n📊 Alle Varianten im Vergleich:")
print(f"{'Daten-Variante':<25} {'ARI':<8} {'Silhouette':<12}")
print("-" * 45)
print(f"{'Ähnliche Skalen':<25} {ari_ideal:.3f}{'':>4} {sil_ideal:.3f}")
print(f"{'Verschiedene (unskal.)':<25} {ari_unscaled:.3f}{'':>4} {sil_unscaled:.3f}")
print(f"{'Verschiedene (skaliert)':<25} {ari_scaled:.3f}{'':>4} {sil_scaled:.3f}")

## 10. Zusammenfassung und Erkenntnisse

Was haben wir gelernt?

In [ ]:
print("🎓 Wichtige Erkenntnisse aus diesem Notebook:")
print()
print("Perfekte Bedingungen für k-Means:")
print("   • Runde, gut getrennte Cluster")
print("   • Ähnlich große Cluster")
print("   • Ähnliche Skalen → k-Means funktioniert perfekt!")
print()
print("⚠️ Was Skalierungsprobleme verursacht:")
print("   • Ein Feature hat viel größere Werte als das andere")
print("   • k-Means fokussiert nur auf das 'große' Feature")
print("   • Das 'kleine' Feature wird fast ignoriert")
print()
print("🔧 Lösung - StandardScaler:")
print("   • Bringt alle Features auf Mittelwert 0, Standardabweichung 1")
print("   • Macht alle Features gleichberechtigt")
print("   • k-Means kann alle Dimensionen fair berücksichtigen")
print()
print("📈 Unsere Ergebnisse:")
print(f"   • Ideale Daten (ähnliche Skalen):  ARI = {ari_ideal:.3f}, Silhouette = {sil_ideal:.3f}")
print(f"   • Problematische Daten (unsk.):     ARI = {ari_unscaled:.3f}, Silhouette = {sil_unscaled:.3f}")
print(f"   • Problematische Daten (skaliert):  ARI = {ari_scaled:.3f}, Silhouette = {sil_scaled:.3f}")
print()
print("🎯 Takeaway:")
print("   • Bei ähnlichen Skalen: Skalierung ist optional")
print("   • Bei verschiedenen Skalen: Skalierung ist KRITISCH!")
print("   • Im Zweifel: Immer skalieren - schadet nie, hilft oft!")